In [ ]:
COMPLEX_GEMINI_MODEL="gemini-2.5-flash"
API_KEY="..."

from vanna.chromadb import ChromaDB_VectorStore
from vanna.google import GoogleGeminiChat

class CustomVanna(ChromaDB_VectorStore, GoogleGeminiChat):
    
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(
            self, 
            config=config
        )
        GoogleGeminiChat.__init__(
            self, 
            config={
                'api_key': API_KEY, 
                'model_name': COMPLEX_GEMINI_MODEL
            }
        )

    def generate_query_explanation(self, sql: str):
        my_prompt = [
            self.system_message("You are a helpful assistant that will explain a SQL query"),
            self.user_message("Explain this SQL query: " + sql),
        ]
        return self.submit_prompt(prompt=my_prompt)


# Initialize Vanna instance globally
dataAgent = CustomVanna({"path":r"C:\Users\Lim Fang Wei\Downloads\personal\data_agent\chroma_path"})
# dataAgent.connect_to_mssql(
#     odbc_conn_str='DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost,54180;DATABASE=historian;UID=n8n;PWD=password'
# )

dataAgent.connect_to_postgres(
    host="localhost",
    dbname="historian",
    user="postgres",
    password="password",
    port=5432,
)

c:\Users\Lim Fang Wei\Downloads\personal\data_agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataAgent.train(ddl="""
                    CREATE TABLE IF NOT EXISTS public.historian
                    (
                        "tagname" text COLLATE pg_catalog."default",
                        "datetime" text COLLATE pg_catalog."default",
                        "value" double precision,
                        "vvalue" double precision,
                        "minraw" double precision,
                        "maxraw" double precision,
                        "mineu" double precision,
                        "maxeu" double precision,
                        "unit" text COLLATE pg_catalog."default",
                        "quality" boolean,
                        "qualitydetail" bigint,
                        "qualitystring" text COLLATE pg_catalog."default",
                        "wwresolution" bigint,
                        "startdatetime" timestamp without time zone
                    )
                """)

Adding ddl: 
                    CREATE TABLE IF NOT EXISTS public.historian
                    (
                        "TagName" text COLLATE pg_catalog."default",
                        "DateTime" text COLLATE pg_catalog."default",
                        "Value" double precision,
                        "vValue" double precision,
                        "MinRaw" double precision,
                        "MaxRaw" double precision,
                        "MinEU" double precision,
                        "MaxEU" double precision,
                        "Unit" text COLLATE pg_catalog."default",
                        "Quality" boolean,
                        "QualityDetail" bigint,
                        "QualityString" text COLLATE pg_catalog."default",
                        "wwResolution" bigint,
                        "StartDateTime" timestamp without time zone
                    )
                


'18882a5a-7bcf-5ea7-8768-051297475e88-ddl'

In [16]:
df_information_schema = dataAgent.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = dataAgent.get_training_plan_generic(df_information_schema)
# plan

# If you like the plan, then uncomment this and run it to train
dataAgent.train(plan=plan)

In [ ]:
dataAgent.train(
    question="Compare LOOP_2_SP vs LOOP_2_PV for the past 30 minutes.", 
    sql="""Select ""startdatetime"", ""tagname"", ""value"" FROM public.historian WHERE ""tagname"" IN ('Cluster1.LOOP_2_SP','Cluster1.LOOP_2_PV') AND ""startdatetime"" > now() - interval '30 minutes' ORDER BY ""startdatetime";"""
)

'017673f0-a42a-53ce-82e6-dc443ff751ca-sql'

In [ ]:
dataAgent.train(
    question="What is the total kWh consumed by mixers vs pumps this month?", 
    sql="""SELECT CASE WHEN ""tagname"" ILIKE '%Mixer%' THEN 'Mixer' ELSE 'Pump' END AS equipment_type, SUM(""value"") AS total_kWh FROM public.historian WHERE (""tagname"" ILIKE '%Mixer%TotkWh%' OR ""tagname"" ILIKE '%Pump%TotkWh%') AND ""startdatetime"" >= date_trunc('month', current_date) GROUP BY equipment_type;"""
)

'26de079f-48c6-5555-abb2-de674bfc77e4-sql'

In [ ]:
dataAgent.train(
    question="What is the entry speed and accumulator length for today's shift?", 
    sql="""SELECT ""tagname"", AVG(""value"") AS avg_val FROM public.historian WHERE ""tagname"" IN ('Cluster1.EntrySpeed','Cluster1.EntryAccumulatorLength') AND ""startdatetime"" > date_trunc('day', now()) GROUP BY ""tagname"";"""
)

'1bb8c215-20e0-56b2-8d83-1aa7f804d796-sql'

In [ ]:
dataAgent.train(
    question="Was the agitator in Tank 1 overloaded yesterday?", 
    sql="""SELECT MAX(""value"") AS max_power FROM public.historian WHERE ""tagname"" ILIKE '%Bottler_Tank1_Agitator%TotW1S%' AND ""startdatetime""::date = current_date - interval '1 day';"""
)

'37895e38-c198-556a-9083-940126eef756-sql'

In [ ]:
dataAgent.train(
    question="Did Line 1 meet its setpoint speed yesterday?", 
    sql="""SELECT AVG(CASE WHEN ""tagname""='Cluster1.LOOP_3_SP' THEN ""value"" END) AS avg_sp, AVG(CASE WHEN ""tagname""='Cluster1.LOOP_3_PV' THEN ""value"" END) AS avg_pv FROM public.historian WHERE (""tagname""='Cluster1.LOOP_3_SP' OR ""tagname""='Cluster1.LOOP_3_PV') AND ""startdatetime""::date = current_date - interval '1 day';"""
)

'36c84cbf-fb70-5680-a7f9-acb154cbcf4f-sql'

In [ ]:
dataAgent.train(
    question="How much did energy tariffs cost for Cluster1 today?", 
    sql="""SELECT SUM(""value"") AS tariff_cost FROM public.historian WHERE ""tagname""='Cluster1.Tariff' AND  ""startdatetime""::date=current_date;"""
)

'c6180c52-55c6-5d4a-93de-72e0ee07e8d8-sql'

In [ ]:
dataAgent.train(
    question="Forecast total energy cost for this week.", 
    sql="""SELECT SUM(""value"") AS forecast_kWh FROM public.historian WHERE ""tagname"" ILIKE '%TotkWh%' AND ""startdatetime"" > date_trunc('week', current_date);"""
)

'93ca10ed-2f40-5169-9566-065fc5853bdd-sql'

In [ ]:
dataAgent.train(
    question="Suggest best schedule to minimize electricity tariff cost.", 
    sql="""SELECT EXTRACT(HOUR FROM ""startdatetime"") AS hour, AVG(""value"") AS avg_tariff FROM public.historian WHERE""tagname""='Cluster1.Tariff' AND ""startdatetime"" > now() - interval '7 days' GROUP BY hour ORDER BY avg_tariff;"""
)

'b907d564-406a-54cf-ad65-53adae82bf51-sql'

In [ ]:
dataAgent.train(
    question="Show me the real kWh usage of Line 1 pasteurizer in the last 8 hours.", 
    sql="""SELECT "startdatetime", "value" FROM public.historian WHERE "tagname" ILIKE '%PLT_LINE1%TotkWh%' AND "startdatetime" > now() - interval '8 hours' ORDER BY "startdatetime";"""
)

'72380a7a-50ed-57a7-a9bc-05d8041ed63b-sql'

In [ ]:
dataAgent.train(
    question="What is the current power consumption of Raw Skim Milk Out Pump?", 
    sql="""SELECT "startdatetime", "value" FROM public.historian WHERE "tagname" ILIKE '%Raw_SkimMilkOutPump%TotW1S%' ORDER BY "startdatetime" DESC LIMIT 1;"""
)

'd5c1ebb5-9db3-5bb3-a6cd-c33cd41a2801-sql'

In [ ]:
dataAgent.train(
    question="How much power is the Mixer_RawMilk consuming at this moment?", 
    sql="""SELECT "startdatetime", "value" FROM public.historian WHERE "tagname" ILIKE '%Mixer_RawMilk%TotW1S%' ORDER BY "startdatetime" DESC LIMIT 1;"""
)


'04ec502d-64ab-5949-8eac-954443eff9c1-sql'

In [ ]:
dataAgent.train(
    question="HWhat is the total kWh consumed by mixers vs pumps this month?", 
    sql="""SELECT CASE WHEN 'tagname' ILIKE '%Mixer%' THEN 'Mixer' ELSE 'Pump' END AS equipment_type, SUM("value") AS total_kWh FROM public.historian WHERE ("tagname" ILIKE '%Mixer%TotkWh%' OR "tagname" ILIKE '%Pump%TotkWh%');"""
)


'1695f2af-8865-5add-a804-478067889236-sql'

In [8]:
dataAgent.train(
    question="What was the overall OEE for yesterday?", 
    sql="""SELECT AVG(oee) AS overall_oee FROM public.oee_date WHERE date = (CURRENT_DATE - INTERVAL '1 day')::DATE;"""
)


'6175e261-1e9a-54fc-815a-540a85f2b9f5-sql'

In [9]:
dataAgent.train(
    question="Predict today’s milk output.", 
    sql="""WITH rate AS (
            SELECT 
                SUM(qty_output - qty_defect) / (SUM(net_operation_time) / 3600) AS units_per_hr 
            FROM oee_date 
            WHERE date = CURRENT_DATE
        ),
        current_output AS (
            SELECT 
                COALESCE(SUM(qty_output - qty_defect), 0) AS total_good_units
            FROM oee_date
            WHERE date = CURRENT_DATE
        )
        SELECT 
            current_output.total_good_units + 
            (rate.units_per_hr * (24 - EXTRACT(HOUR FROM CURRENT_TIMESTAMP))) AS forecast_output 
        FROM rate, current_output;"""
)


'5c0532ee-3520-5823-aa58-403c96ca7bc8-sql'

In [ ]:
dataAgent.train(
    question="How many hours did Line 1 and Line 2 run this week?", 
    sql="""SELECT 
        id_machine, 
            SUM(net_operation_time) / 3600 AS run_hours 
        FROM oee_date 
        WHERE date >= date_trunc('week', CURRENT_DATE) 
            AND id_machine IN (1, 2) 
        GROUP BY id_machine;
        """
)


'990fdf57-f3f1-5135-816c-356c7a062d1b-sql'

In [11]:
dataAgent.train(
    question="How many hours did Line 1 run this week?", 
    sql="""SELECT 
            SUM(net_operation_time) / 3600 AS run_hours 
        FROM oee_date 
        WHERE id_machine = 1 
            AND date >= date_trunc('week', CURRENT_DATE);
        """
)


'29290990-ebd4-5aac-8fab-365c9d039d3e-sql'

In [ ]:
dataAgent.train(
    question="Show trend of energy consumption vs production output over last 30 days.", 
    sql="""SELECT 
            "startdatetime"::date AS day, 
            SUM("value") AS total_kWh, 
            SUM(od.qty_output - od.qty_defect) AS total_units 
        FROM public.historian r 
        JOIN public.oee_date od ON "startdatetime"::date = od.date 
        WHERE "tagname" ILIKE '%TotkWh%' 
            AND "startdatetime" > CURRENT_TIMESTAMP - INTERVAL '30 days' 
        GROUP BY day 
        ORDER BY day;
        """
)


'10258d8e-22ff-5561-8d7c-18510c0577bb-sql'

In [ ]:
dataAgent.train(
    question="Which pasteurizers are running?", 
        sql="""
        SELECT e.name,
        MAX(""value"") FILTER (WHERE ""tagname"" ILIKE '%state%') AS state
        FROM public.equipment_table e
        JOIN public.historian 
        ON ""tagname"" ILIKE '%' || e.name || '%'
        WHERE e.type = 'pasteurizer'
        AND ""startdatetime"" > now() - interval '5 minutes'
        GROUP BY e.name;

        """
)


'44e97995-f31d-5267-a9d6-f3afbff05700-sql'

In [ ]:
dataAgent.train(
    question="Which bottling line conveyors are running right now?", 
        sql="""
        SELECT DISTINCT e.name, ""value""
        FROM public.equipment_table e
        JOIN public.historian r
        ON ""tagname"" ILIKE '%' || e.name || '%'
        WHERE e.type = 'conveyor'
        AND ""tagname"" ILIKE '%TotW1S%'
        AND ""startdatetime"" > now() - interval '5 minutes'
        AND ""value"" > 0;
        """
)


'335c32b8-8fda-5dd4-95c7-41293e6f320d-sql'

In [4]:
dataAgent.train(
    question="Show the last 5 alarms for Line 2.", 
        sql="""
        SELECT e.name,
            ev.ts,
            ev.severity,
            ev.code,
            ev.message
        FROM event_table ev
        JOIN equipment_table e
        ON ev.equipment_id = e.equipment_id
        WHERE e.line_id = 2
        AND ev.category = 'alarm'
        ORDER BY ev.ts DESC
        LIMIT 5;
        """
)


'8baf07cc-f64e-5a8a-8d2e-8ebe05a40c44-sql'

In [5]:
dataAgent.train(
    question="When was the last CIP cycle completed for Line 1?", 
        sql="""
        SELECT MAX(ev.ts) AS last_cip
        FROM event_table ev
        JOIN equipment_table e
        ON ev.equipment_id = e.equipment_id
        WHERE e.line_id = 1
        AND ev.category = 'cip_end';
        """
)



'e9471a87-3760-5f16-9e49-c7fbc8fcb807-sql'

In [6]:
dataAgent.train(
    question="Why did the bottler filler stop in the last hour?", 
        sql="""
        SELECT 
            ev.ts,
            ev.severity,
            ev.message
        FROM event_table ev
        JOIN equipment_table e
        ON ev.equipment_id = e.equipment_id
        WHERE e.name ILIKE '%Bottler_Filler%'
        AND ev.category = 'stop'
        AND ev.ts > now() - interval '1 hour'
        ORDER BY ev.ts;

        """
)



'38637868-7d17-5838-bd48-b66d0296a849-sql'

In [ ]:
dataAgent.train(
    question="List top 3 equipment by power consumption in the last 24 hours.", 
        sql="""
        SELECT e.name, SUM("value") AS total_kWh
        FROM equipment_table e
        JOIN public.historian r
        ON "tagname" ILIKE '%' || e.name || '%'
        WHERE "tagname" ILIKE '%TotkWh%'
        AND "startdatetime" > now() - interval '24 hours'
        GROUP BY e.name
        ORDER BY total_kWh DESC
        LIMIT 3;

        """
)



'9160f950-a0e1-5a7f-8cdc-0709ea7f9832-sql'

In [8]:
dataAgent.train(
    question="How much downtime came from CIP vs maintenance last week?", 
        sql="""
WITH CalculatedDowntime AS (
    SELECT
        ev.category,
        -- Calculate the duration (in minutes) from the current event to the next one
        EXTRACT(EPOCH FROM (
            LEAD(ev.ts) OVER (ORDER BY ev.ts) - ev.ts
        )) / 60 AS duration_minutes
    FROM
        event_table ev
    WHERE
        -- Filter for the event categories that define 'downtime'
        ev.category IN ('cip', 'maintenance', 'stop')
        -- Filter for events within the last 7 days (assuming 'now()' is the current time)
        AND ev.ts > now() - interval '7 days' 
)
SELECT
    category,
    -- Now, sum the pre-calculated duration_minutes for each category
    SUM(duration_minutes) AS total_downtime_minutes
FROM
    CalculatedDowntime
GROUP BY
    category;

        """
)



'b1cfd328-6b28-5d14-98f3-892204a62e8a-sql'

In [9]:
dataAgent.train(
    question="Export alarms and events for BLD_PM800 for audit.", 
        sql="""
SELECT ev.ts,
       ev.severity,
       ev.code,
       ev.message
FROM event_table ev
JOIN equipment_table e
  ON ev.equipment_id = e.equipment_id
WHERE e.name ILIKE '%BLD_PM800%'
  AND ev.ts > now() - interval '7 days'
ORDER BY ev.ts;

        """
)



'f30e6d78-fdc1-5112-870f-fa956e457824-sql'

In [1]:
# Read markdown file into string
with open('equipment_mapping.md', 'r', encoding='utf-8') as f:
    md_content = f.read()

print(md_content)

# Equipment and Tag Mapping for Historian Data

Each production unit or device in the milk plant corresponds to a tag pattern in the Historian table:

| Equipment Name | Tag Pattern | Description |
|-----------------|-------------|--------------|
| Skim milk outlet pump | Cluster1.Raw_SkimMilkOutPump_PM800% | Measures power/energy for the skim milk outlet pump |
| Full milk outlet pump | Cluster1.Raw_FullMilkOutPump_PM800% | Power and kWh usage for full milk outlet pump |
| Line 1 pasteurization unit | Cluster1.PLT_LINE1_PM800% | Represents pasteurizer line 1 (called 'PLT' in tags) |
| Line 2 pasteurization unit | Cluster1.PLT_LINE2_PM800% | Represents pasteurizer line 2 |
| Filling machine | Cluster1.Bottler_Filler_PM800% | Filling station power and kWh tags |
| Bottle labeling machine | Cluster1.Bottler_Label_PM800% | Labeling machine tags |
| Conveyor between filler and labeler | Cluster1.Bottler_Conveyor_PM800% | Conveyor line equipment |
| Bottle capping machine | Cluster1.Bottler

In [ ]:
# Read markdown file into string
with open('mapping.json', 'r', encoding='utf-8') as f:
    json_content = f.read()

print(json_content)

{
  "equipment_mapping": {
    "skim_milk": {
      "keywords": ["skim milk", "skim", "skimmed milk"],
      "tagnames": [
        "Cluster1.Raw_SkimMilkOutPump_PM800%",
        "Cluster1.Raw_SkimMilkChiller_ON"
      ],
      "description": "Skim milk processing equipment including pump and chiller"
    },
    "full_milk": {
      "keywords": ["full milk", "whole milk", "full fat milk"],
      "tagnames": [
        "Cluster1.Raw_FullMilkOutPump_PM800%",
        "Cluster1.Raw_FullMilkChiller_ON"
      ],
      "description": "Full milk processing equipment including pump and chiller"
    },
    "mixer": {
      "keywords": ["mixer", "mixing", "blend", "agitator"],
      "tagnames": [
        "Cluster1.Mixer_RawMilk_PM800%",
        "Cluster1.Mixer_Mixer_PM800%",
        "Cluster1.Mixer_Agitator_%"
      ],
      "description": "Mixing equipment for raw milk and ingredients"
    },
    "pasteurizer": {
      "keywords": ["pasteurizer", "pasteurization", "heat treatment", "PLT"],
      "

In [ ]:
# Read markdown file into string
with open('document_vanna.csv', 'r', encoding='utf-8') as f:
    csv_content = f.read()

print(csv_content)

tagname,description
Cluster1.Raw_SkimMilkOutPump_PM800%,Skim milk outlet pump
Cluster1.Raw_FullMilkOutPump_PM800%,Full milk outlet pump
Cluster1.Raw_SkimMilkChiller_ON,Skim milk chiller
Cluster1.Raw_FullMilkChiller_ON,Full milk chiller
Cluster1.Mixer_RawMilk_PM800%,Raw milk pre-mixer
Cluster1.Mixer_Mixer_PM800%,Main ingredient mixer
Cluster1.Mixer_Agitator_%,Mixer agitator motor
Cluster1.Mixer_SugarFeed_ON,Sugar feeding system
Cluster1.Mixer_MaltFeed_ON,Malt feeder
Cluster1.Mixer_StrawFeed_ON,Straw ingredient feeder
Cluster1.PLT_LINE1_PM800%,Line 1 pasteurization unit
Cluster1.PLT_LINE2_PM800%,Line 2 pasteurization unit
Cluster1.Bottler_Filler_PM800%,Filling machine
Cluster1.Bottler_Label_PM800%,Bottle labeling machine
Cluster1.Bottler_Conveyor_PM800%,Conveyor between filler and labeler
Cluster1.Bottler_CapConveyor_PM800%,Cap transport conveyor
Cluster1.Bottler_Cap_PM800%,Bottle capping machine
Cluster1.Bottler_Tank1_Agitator_PM800%,Agitator motor for Tank 1
Cluster1.Bottler_Tank2_Agit

In [ ]:
dataAgent.add_documentation(md_content)
dataAgent.add_documentation(json_content)
dataAgent.add_documentation(csv_content)

In [7]:
dataAgent.get_training_data()

,id,question,content,training_data_type
0,96954093-e2e7-520f-b04b-6cef57836d4e-sql,what are the tables available,"SELECT table_schema, table_name\nFROM informat...",sql
1,26de079f-48c6-5555-abb2-de674bfc77e4-sql,What is the total kWh consumed by mixers vs pu...,"SELECT CASE WHEN """"TagName"""" ILIKE '%Mixer%' T...",sql
2,1bb8c215-20e0-56b2-8d83-1aa7f804d796-sql,What is the entry speed and accumulator length...,"SELECT """"TagName"""", AVG(""""Value"""") AS avg_val ...",sql
3,37895e38-c198-556a-9083-940126eef756-sql,Was the agitator in Tank 1 overloaded yesterday?,"SELECT MAX(""""Value"""") AS max_power FROM public...",sql
4,36c84cbf-fb70-5680-a7f9-acb154cbcf4f-sql,Did Line 1 meet its setpoint speed yesterday?,"SELECT AVG(CASE WHEN """"TagName""""='Cluster1.LOO...",sql
...,...,...,...,...
207,b028d91f-e4af-568e-999c-d7f5bea4f40b-doc,None,The following columns are in the role_routine_...,documentation
208,fb6b65e6-6742-52d7-b66b-5bc4e12a9609-doc,None,The following columns are in the routine_routi...,documentation
209,594e2e57-c122-587d-b351-53b64b0d5682-doc,None,The following columns are in the routine_seque...,documentation
210,7aff8cc2-dc50-5ee5-b918-4bcc84a1b9e5-doc,None,The following columns are in the routine_table...,documentation
